In [1]:
import numpy as np #for numerical computations
import pandas as pd #for dataframe operations

from matplotlib import pyplot as plt #for viewing images and plots
%matplotlib inline 
#So that Matplotlib plots don't open in separate windows outside the notebook

import urllib #For fetching data from Web URLs

import cv2   #For image processing

from sklearn.preprocessing import LabelEncoder    #For encoding categorical variables
from sklearn.model_selection import train_test_split #For splitting of dataset

#All tensorflow utilities for creating, training and working with a CNN
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, BatchNormalization
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
objek = 'hand'

In [3]:
def prepare_data(file_name):
    df = pd.read_csv(r'{}'.format(file_name), names=["path", "target"])
    print(df["path"].head())
    np.where(df.applymap(lambda x: x == ''))
    
    # X Input Data
    image_list = []

    for path in df['path'] :
#         print(path) 
        image_arr = cv2.imread(path)
        image_224x224 = cv2.resize(image_arr, (224, 224))
        image_list.append(image_224x224.astype('float')) 
    X = np.array(image_list)
    print("Total Data = ", X.shape)

    # Y Output Data
    encoder = LabelEncoder()
    Targets = encoder.fit_transform(df['target'])
    n_classes = 2
    Y = to_categorical(Targets, num_classes = n_classes)
    
    return X, Y

In [4]:
X, Y = prepare_data("{}_with_label_randomized.csv".format(objek))
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.15, random_state=13)
print("Data Train", X_train.shape)
print("Data Val", X_val.shape)

0    MURA-v1.1/train/XR_HAND/patient10169/study1_ne...
1    MURA-v1.1/train/XR_HAND/patient10121/study1_po...
2    MURA-v1.1/train/XR_HAND/patient09111/study1_ne...
3    MURA-v1.1/train/XR_HAND/patient11008/study1_ne...
4    MURA-v1.1/train/XR_HAND/patient09780/study1_po...
Name: path, dtype: object
Total Data =  (5543, 224, 224, 3)
Data Train (4711, 224, 224, 3)
Data Val (832, 224, 224, 3)


In [5]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import layers 
from tensorflow.keras import Model 
import tensorflow as tf 

base_model = VGG16(input_shape = (224, 224, 3), # Shape of our images
include_top = False, # Leave out the last fully connected layer
weights = 'imagenet')
for layer in base_model.layers:
    layer.trainable = False
# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final sigmoid layer with 1 node for classification output
x = layers.Dense(2, activation='softmax')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = tf.keras.optimizers.RMSprop(lr=0.0001), loss = 'binary_crossentropy',metrics = ['acc'])

In [6]:
save_at = "VGG_Model/vgg_{}.h5".format(objek)
save_best = ModelCheckpoint (save_at, monitor='val_acc', verbose=0, save_best_only=True, save_weights_only=False, mode='max')
vgghist = model.fit(X_train, Y_train, validation_data = (X_val, Y_val), callbacks=[save_best], batch_size=256, epochs = 50)

Train on 4711 samples, validate on 832 samples
Epoch 1/50
4711/4711 [==============================] - 26s 5ms/sample - loss: 4.1443 - acc: 0.7173 - val_loss: 3.9557 - val_acc: 0.7428
Epoch 2/50
4711/4711 [==============================] - 14s 3ms/sample - loss: 4.1459 - acc: 0.7304 - val_loss: 3.9557 - val_acc: 0.7428
Epoch 3/50
4711/4711 [==============================] - 14s 3ms/sample - loss: 4.1459 - acc: 0.7304 - val_loss: 3.9557 - val_acc: 0.7428
Epoch 4/50
4711/4711 [==============================] - 14s 3ms/sample - loss: 4.1459 - acc: 0.7304 - val_loss: 3.9557 - val_acc: 0.7428
Epoch 5/50
4711/4711 [==============================] - 14s 3ms/sample - loss: 4.1459 - acc: 0.7304 - val_loss: 3.9557 - val_acc: 0.7428
Epoch 6/50
4711/4711 [==============================] - 14s 3ms/sample - loss: 4.1459 - acc: 0.7304 - val_loss: 3.9557 - val_acc: 0.7428
Epoch 7/50
4711/4711 [==============================] - 14s 3ms/sample - loss: 4.1459 - acc: 0.7304 - val_loss: 3.9557 - val_acc: 0

KeyboardInterrupt: 

In [ ]:
# model.save('vgg_{}_81_float.h5'.format(objek))

In [ ]:
X_test, Y_test = prepare_data("{}_val_path_label.csv".format(objek))
# X_test, Y_test = prepare_data("finger_val_path_label.csv")

In [ ]:
model_load = load_model("VGG_Model/vgg_{}.h5".format(objek))
score = model_load.evaluate(X_test, Y_test, verbose=0)
print('Accuracy over the test set: \n ', round((score[1]*100), 2), '%')

In [ ]:
Y_pred = model_load.predict(X_test)

In [ ]:
X_test.shape

In [ ]:
np.random.seed(87)
for rand_num in np.random.randint(0, len(Y_test), 15):
  plt.figure()
  plt.imshow(X_test[rand_num].astype('int')), plt.axis('off')
  if np.where(Y_pred[rand_num] == 1)[0].sum() == np.where(Y_test[rand_num] == 1)[0].sum():
    plt.title(encoder.classes_[np.where(Y_pred[rand_num] == 1)[0].sum()], color='g')
  else :
    plt.title(encoder.classes_[np.where(Y_pred[rand_num] == 1)[0].sum()], color='r')